In [ ]:
!pip install selenium pillow pytesseract webdriver-manager pandas tqdm telegram



In [16]:
from datetime import datetime

# Get the current date and time
current_datetime = datetime.now()

# Format the date and time
formatted_datetime = current_datetime.strftime("%Y-%m-%d %H:%M:%S")

print("Current date and time:", formatted_datetime)


Current date and time: 2025-01-03 00:08:38


# improved

In [17]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, StaleElementReferenceException
from PIL import Image, ImageEnhance, ImageFilter
import pytesseract
from io import BytesIO
from webdriver_manager.chrome import ChromeDriverManager
import time
import re
import pandas as pd  # Import pandas
from tqdm import tqdm  # Import tqdm for progress bars
import concurrent.futures
from datetime import datetime, timedelta

# Function to solve CAPTCHA with retry logic
def solve_captcha(driver, image_xpath):
    while True:  # Keep trying to solve CAPTCHA indefinitely until successful.
        try:
            captcha_element = driver.find_element(By.XPATH, image_xpath)
            captcha_image = captcha_element.screenshot_as_png
            captcha_image = Image.open(BytesIO(captcha_image))

            # Preprocess the image to improve OCR accuracy
            captcha_image = captcha_image.convert("L")  # Convert to grayscale
            captcha_image = captcha_image.filter(ImageFilter.MedianFilter(size=3))  # Reduce noise
            enhancer = ImageEnhance.Contrast(captcha_image)
            captcha_image = enhancer.enhance(2)  # Increase contrast
            
            # Use Tesseract to extract text
            captcha_text = pytesseract.image_to_string(captcha_image, config='--psm 6')
            cleaned_text = re.sub(r'[^A-Za-z0-9]', '', captcha_text)  # Remove special characters
            
            # Enter Captcha text into the input field
            captcha_field = driver.find_element(By.XPATH, "//*[@id='captchaText']")
            captcha_field.clear()
            captcha_field.send_keys(cleaned_text)
            
            # Submit the form and check for success or failure.
            submit_button = driver.find_element(By.XPATH, "//*[@id='collapse1']/div[5]/button")
            submit_button.click()
            
            time.sleep(2)  # Allow time for form to process
            
            if not is_captcha_incorrect(driver, "//*[@id='errmsg']"):
                print("Captcha accepted, proceeding...")
                return True  # Captcha solved successfully

        except NoSuchElementException:
            print("CAPTCHA element not found. Retrying...")
        except Exception as e:
            print(f"Error while solving CAPTCHA: {e}")
        
        time.sleep(1)  # Small delay before retrying

def is_captcha_incorrect(driver, error_xpath):
    try:
        error_message = driver.find_element(By.XPATH, error_xpath).text
        if "Captcha Code doesn't Match" in error_message:
            print("Detected Captcha error: Code doesn't match. Retrying...")
            return True
    except NoSuchElementException:
        pass
    return False

# Function to process each region code with improved handling for stale elements and timeouts.
def process_region(region_code):
    options = Options()
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--headless")

    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)

    all_region_data = []

    try:
        url = "https://www.fois.indianrail.gov.in/FOISWebPortal/pages/FWP_ODROtsgDtls.jsp"
        
        driver.get(url)
        print(f"\nPage Loaded for region: {region_code}")

        wait = WebDriverWait(driver, 120)  # Wait indefinitely for elements to load
        
        outstanding_odr_option = wait.until(EC.presence_of_element_located((By.ID, "Zone")))
        outstanding_odr_option.click()
        outstanding_odr_option.send_keys(region_code)
        print(f"Selected '{region_code}' from the dropdown.")

        captcha_image_xpath = "/html/body/div[4]/center/form/div/div[2]/div[4]/img[1]"
        
        if not solve_captcha(driver, captcha_image_xpath):
            raise Exception(f"Unable to solve Captcha for region {region_code} after multiple attempts.")

        print("Waiting for iframe to load...")
        
        data_div = wait.until(EC.presence_of_element_located((By.XPATH, "//*[@id='dataDiv']")))
        
        iframe = data_div.find_element(By.TAG_NAME, "iframe")
        driver.switch_to.frame(iframe)
        
        print("Waiting for the table to load...")
        
        table_element = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, "body > div > table")))
        
        tbody_element = table_element.find_element(By.TAG_NAME, "tbody")
        
        rows = tbody_element.find_elements(By.TAG_NAME, "tr")

        region_data_list = []

        date = (datetime.now() - timedelta(days=1)).strftime("%d-%m-%y")
        # date = time.strftime("%d-%m-%y")  # Get today's date in DD-MM-YY format

        for row in rows:
            columns = [col.text for col in row.find_elements(By.TAG_NAME, "td")]
            
            # Filter rows: Column 10 must be 'FG' or 'DOC' and Column 5 must be today's date
            if len(columns) >= 10 and columns[9] in ["FG", "DOC"] and columns[4] == date:
                print(columns)  # Print each row's data for debugging purposes.
                region_data_list.append(columns)

            elif len(columns) < 23:
                print(f"Skipping row with insufficient columns: {columns}")

            elif not any(columns):
                print("Skipping empty row.")

        if not region_data_list:
            print(f"No valid data found for region {region_code}. Skipping this region.")
            return None

        column_names = [
            "S.No.", "DVSN", "STTN FROM", "DEMAND NO.", 
            "DEMAND DATE", "DEMAND TIME", "Expected loading date", 
            "CNSR", "CNSG", "CMDT", 
            "TT", "PC", "PBF", 
            "VIA", "RAKE CMDT", "DSTN", 
            "INDENTED TYPE", "INDENTED UNTS", 
            "INDENTED 8W", "OTSG UNTS", 
            "OTSG 8W", "SUPPLIED UNTS", 
            "SUPPLIED TIME"
        ]

        df_region = pd.DataFrame(region_data_list, columns=column_names)
        df_region['Region'] = region_code  

        return df_region

    except Exception as e:
        print(f"An error occurred while processing region {region_code}: {e}")
        return None

    finally:
        driver.quit()

# List of region codes to process
# region_codes = ["SW"]
region_codes = [
    "CR", "DFCR", "EC", "ECO", "ER", 
    "KR", "NC", "NE", "NF", "NPLR", 
    "NR", "NW", "SC", "SE", "SEC", 
    "SR", "SW", "WC", "WR"
]

# Combine all region DataFrames into a single DataFrame using parallel processing
all_regions_data = []

with concurrent.futures.ThreadPoolExecutor() as executor:
    results = list(tqdm(executor.map(process_region, region_codes), total=len(region_codes), desc="Processing Regions"))

# Filter out None results and concatenate DataFrames into a single DataFrame and save it.
results = [df for df in results if df is not None]

if results:
    final_df = pd.concat(results, ignore_index=True)
    
    print(final_df.head())  # Display first few rows of the final DataFrame.
    
    final_df.to_csv('output_combined_regions_daily.csv', index=False)

    # Count and print the number of rows after filtering
    filtered_rows_count = len(final_df)
    print(f"Total rows with FG or DOC and today's date: {filtered_rows_count}")
else:
    print("No data collected from any regions.")

final_df_without_FCI = final_df[final_df['CNSR'] != "FCI"]
final_df_without_FCI.to_csv('final_df_without_FCI.csv', index=False)

print("final_df_without_FCI saved")

# Read dropdown options data
options_df = pd.read_csv('dropdown_options.csv')
print("options_df opened successfully")

# Extract short forms and full forms
options_df['Short_Form'] = options_df['Current_Stations'].str.extract(r'\((.*?)\)')
options_df['Full_Form'] = options_df['Current_Stations'].str.replace(r'\(.*?\)', '', regex=True).str.strip()

# Create mapping dictionary
short_form_dict_stn = dict(zip(options_df['Short_Form'], options_df['Full_Form']))

# Replace short forms in 'STTN FROM' and 'DSTN'
final_df_without_FCI['STTN FROM'] = final_df_without_FCI['STTN FROM'].map(short_form_dict_stn).fillna(final_df_without_FCI['STTN FROM'])
final_df_without_FCI['DSTN'] = final_df_without_FCI['DSTN'].map(short_form_dict_stn).fillna(final_df_without_FCI['DSTN'])

# Consignee mappings
consignee_consiner_df = pd.read_csv('consignee_consiner_data.csv')
print("consignee_consiner_df opened successfully")

consignee_consiner_df['Short_Form'] = consignee_consiner_df['Consignee Name'].str.extract(r'\((.*?)\)')
consignee_consiner_df['Full_Form'] = consignee_consiner_df['Consignee Name'].str.replace(r'\(.*?\)', '', regex=True).str.strip()

short_form_dict = dict(zip(consignee_consiner_df['Short_Form'], consignee_consiner_df['Full_Form']))

final_df_without_FCI['CNSR_Full'] = final_df_without_FCI['CNSR'].map(short_form_dict)
final_df_without_FCI['CNSG_Full'] = final_df_without_FCI['CNSG'].map(short_form_dict)

# Mapping DVSN
# Create a mapping dictionary for DVSN short forms to full forms
dvsn_mapping = {
    'BB': 'Bandra Division',
    'BSL': 'Bhopal Division',
    'NGP': 'Nagpur Division',
    'PUNE': 'Pune Division',
    'SUR': 'Surat Division',
    'WDFC': 'Western Dedicated Freight Corridor',
    'DDU': 'Deen Dayal Upadhyaya Junction',
    'DHN': 'Dhanbad Division',
    'DNR': 'Danapur Division',
    'SEE': 'Sealdah Division',
    'SPJ': 'Samastipur Division',
    'KUR': 'Khurda Road Division',
    'SBP': 'Sambalpur Division',
    'WAT': 'Wadi Division',
    'ASN': 'Asansol Division',
    'HWH': 'Howrah Division',
    'MLDT': 'Malda Town Division',
    'KAWR': 'Kalyan Division',
    'RN':  'Rourkela Division',
    'AGRA':  "Agra Division",
    'JHS':  "Jhansi Division",
    'PRYJ':  "Prayagraj Junction",
    'BSB':  "Varanasi Division",
    'IZN':  "Izatnagar Division",
    'LJN':  "Lucknow Junction",
    'APDJ':  "Amritsar Division",
    'KIR':  "Kharagpur Division",
    'LMG':  "Ludhiana Division",
    'RNY':  "Rani Kamlapati Division",
    'TSK':  "Tinsukia Division",
    'DLI':  "Delhi Division",
    'FZR':  "Ferozepur Division",
    'LKO':  "Lucknow Division",
    'MB':   "Moradabad Division",
    'UMB':  "Ambala Division",
    'AII':  "Ajmer Division",
    'BKN':  "Bikaner Division",
    'JP':   "Jaipur Division",
    'JU':   "Jodhpur Division",
    'BZA':  "Vijayawada Division",
    'GNT':  "Guntur Division",
    'GTL':  "Guntakal Junction",
    'HYB':  "Hyderabad Division",
    'NED':  "Nanded Division",
    "ADRA": "Adra Junction",
    "CKP": "Chhapra Kacheri",
    "KGP": "Kharagpur",
    "RNC": "Ranchi",
    "BSP": "Bilaspur",
    "NAG": "Nagaur",
    "R": "Rourkela",
    "MAS": "Chennai Egmore (Madras)",
    "MDU": "Madurai",
    "SA": "Salem",
    "TPJ": "Tiruchirappalli Junction",
    "TVC": "Thiruvananthapuram Central",
    "MYS": "Mysuru",
    "SBC": "Krantivira Sangolli Rayanna (Bangalore City)",
    "UBL": "Hubli",
    "BPL": "Bhopal",
    "JBP": "Jabalpur",
    "KOTA": "Kota",
    "ADI": "Ahmedabad",
    "BCT": "Bhavnagar Terminus",
    "BRC": "Vadodara Junction (Baroda)",
    "BVC": "Bhopal (BVC)",
    "RJT": "Rajkot Junction",
    "RTM": "Ratlam Junction"
}

final_df_without_FCI['DVSN'] = final_df_without_FCI['DVSN'].replace(dvsn_mapping)

final_df_without_FCI.to_csv('final_df_without_FCI.csv', index=False)
print("Final file has been generated.")


Processing Regions:   0%|          | 0/19 [00:00<?, ?it/s]


Page Loaded for region: ER

Page Loaded for region: NW

Page Loaded for region: SC

Page Loaded for region: KR
Selected 'ER' from the dropdown.

Page Loaded for region: ECO

Page Loaded for region: NR

Page Loaded for region: CR

Page Loaded for region: NF
Selected 'ECO' from the dropdown.

Page Loaded for region: NC
Selected 'NF' from the dropdown.

Page Loaded for region: DFCR
Selected 'DFCR' from the dropdown.
Selected 'NC' from the dropdown.
Selected 'KR' from the dropdown.
Selected 'CR' from the dropdown.
Selected 'NW' from the dropdown.
Selected 'SC' from the dropdown.

Page Loaded for region: NE
Selected 'NR' from the dropdown.
Detected Captcha error: Code doesn't match. Retrying...
Detected Captcha error: Code doesn't match. Retrying...
Detected Captcha error: Code doesn't match. Retrying...
Selected 'NE' from the dropdown.
Detected Captcha error: Code doesn't match. Retrying...
Captcha accepted, proceeding...
Waiting for iframe to load...
Waiting for the table to load...
Dete

Processing Regions:   5%|▌         | 1/19 [06:07<1:50:23, 367.96s/it]

Detected Captcha error: Code doesn't match. Retrying...
Detected Captcha error: Code doesn't match. Retrying...
Detected Captcha error: Code doesn't match. Retrying...
Detected Captcha error: Code doesn't match. Retrying...
Detected Captcha error: Code doesn't match. Retrying...
Detected Captcha error: Code doesn't match. Retrying...
Detected Captcha error: Code doesn't match. Retrying...
Detected Captcha error: Code doesn't match. Retrying...
Detected Captcha error: Code doesn't match. Retrying...
Captcha accepted, proceeding...
Waiting for iframe to load...
Waiting for the table to load...
['63', 'ASN', 'DUMK', '4', '02-01-25', '12:17', '04-01-2025', 'BRIW', 'BRIW', 'FG', 'GG', 'B', 'N', '', 'BR', 'VZP', 'BCN', '21', '21', '21', '21', '0', '']
Skipping row with insufficient columns: ['TOTAL', '112306', '112306', '111281', '111281', '922', ' ']
No valid data found for region SE. Skipping this region.
Skipping row with insufficient columns: ['TOTAL', '80525', '80525', '80324', '80324',

Processing Regions:  21%|██        | 4/19 [19:18<1:10:50, 283.40s/it]

['5221', 'BSP', 'RIG', '1', '02-01-25', '19:59', '03-01-2025', 'FCI', 'FCI', 'FG', 'GG', 'B', 'N', '', 'R', 'KBL', 'BCN', '42', '42', '42', '42', '0', '']
['5679', 'NAG', 'DGG', '1', '02-01-25', '14:50', '03-01-2025', 'SGRI', 'SGRI', 'FG', 'GG', 'B', 'N', '', 'R', 'KNGK', 'BCN', '42', '42', '42', '42', '0', '']
['5715', 'NAG', 'LIG', '1', '02-01-25', '09:10', '02-01-2025', 'SYJT', 'SYJT', 'FG', 'GG', 'B', 'N', '', 'M', 'IDBR', 'BCN', '42', '42', '42', '42', '0', '']
['5724', 'NAG', 'RDPS', '1', '02-01-25', '19:25', '03-01-2025', 'FCI', 'FCI', 'FG', 'GG', 'B', 'N', '', 'R', 'MDP', 'BCN', '42', '42', '42', '42', '0', '']
['5795', 'R', 'RSD', '1', '02-01-25', '20:08', '02-01-2025', 'SHMI', 'SHMI', 'FG', 'GG', 'B', 'N', '', 'BR', 'KNGK', 'BCNHL', '58', '58', '58', '58', '0', '']
['5797', 'R', 'TLD', '1', '02-01-25', '23:19', '03-01-2025', 'FCI', 'FCI', 'FG', 'GG', 'B', 'N', '', 'R', 'TJSP', 'BCN', '42', '42', '42', '42', '0', '']
Skipping row with insufficient columns: ['TOTAL', '334920', 

Processing Regions: 100%|██████████| 19/19 [37:31<00:00, 118.50s/it] 

  S.No. DVSN STTN FROM DEMAND NO. DEMAND DATE DEMAND TIME  \
0    93  BSL       BGN          1    02-01-25       11:20   
1    63  ASN      DUMK          4    02-01-25       12:17   
2     8  IZN        PP          1    02-01-25       18:41   
3     9  IZN        PP          1    02-01-25       18:41   
4    10  IZN        PP          1    02-01-25       18:41   

  Expected loading date  CNSR  CNSG CMDT  ... RAKE CMDT  DSTN INDENTED TYPE  \
0            04-01-2025  DAYE  DAYE  DOC  ...       DOC   POY           BCN   
1            04-01-2025  BRIW  BRIW   FG  ...        BR   VZP           BCN   
2            04-01-2025   RLE  MULT   FG  ...         W  SGWF           BCN   
3            04-01-2025   RLE  MULT   FG  ...         W  SGWF           BCN   
4            04-01-2025   RLE  MULT   FG  ...         W  SGWF           BCN   

  INDENTED UNTS INDENTED 8W OTSG UNTS OTSG 8W SUPPLIED UNTS SUPPLIED TIME  \
0            21          21        21      21             0                 
1   


C:\Users\intel\AppData\Local\Temp\ipykernel_852\1501055881.py:210: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df_without_FCI['STTN FROM'] = final_df_without_FCI['STTN FROM'].map(short_form_dict_stn).fillna(final_df_without_FCI['STTN FROM'])
C:\Users\intel\AppData\Local\Temp\ipykernel_852\1501055881.py:211: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df_without_FCI['DSTN'] = final_df_without_FCI['DSTN'].map(short_form_dict_stn).fillna(final_df_without_FCI['DSTN'])
C:\Users\intel\AppData\Loc

In [18]:
final_df_without_FCI

,S.No.,DVSN,STTN FROM,DEMAND NO.,DEMAND DATE,DEMAND TIME,Expected loading date,CNSR,CNSG,CMDT,...,INDENTED TYPE,INDENTED UNTS,INDENTED 8W,OTSG UNTS,OTSG 8W,SUPPLIED UNTS,SUPPLIED TIME,Region,CNSR_Full,CNSG_Full
0,93,Bhopal Division,BORGAON,1,02-01-25,11:20,04-01-2025,DAYE,DAYE,DOC,...,BCN,21,21,21,21,0,,CR,M/S DAYAL ENERGY AND PROTEINS LTD.,M/S DAYAL ENERGY AND PROTEINS LTD.
1,63,Asansol Division,DUMKA,4,02-01-25,12:17,04-01-2025,BRIW,BRIW,FG,...,BCN,21,21,21,21,0,,ER,M/S BALAJI RICE INDUSTRIES PVT LTD,M/S BALAJI RICE INDUSTRIES PVT LTD
2,8,Izatnagar Division,PURANPUR,1,02-01-25,18:41,04-01-2025,RLE,MULT,FG,...,BCN,5,5,5,5,0,,NE,M/S R L ENTERPRISES,NaN
3,9,Izatnagar Division,PURANPUR,1,02-01-25,18:41,04-01-2025,RLE,MULT,FG,...,BCN,6,6,6,6,0,,NE,M/S R L ENTERPRISES,NaN
4,10,Izatnagar Division,PURANPUR,1,02-01-25,18:41,04-01-2025,RLE,MULT,FG,...,BCN,11,11,11,11,0,,NE,M/S R L ENTERPRISES,NaN
5,11,Izatnagar Division,PURANPUR,1,02-01-25,18:41,04-01-2025,RLE,MULT,FG,...,BCN,20,20,20,20,0,,NE,M/S R L ENTERPRISES,NaN
15,26,Bikaner Division,SHRI VIJAI NAGAR,1,02-01-25,12:58,03-01-2025,DPIM,DPIM,FG,...,BCN,21,21,21,21,0,,NW,M/S D P IMPEX,M/S D P IMPEX
16,27,Bikaner Division,SHRI VIJAI NAGAR,2,02-01-25,12:58,03-01-2025,DPTR,DPTR,FG,...,BCN,21,21,21,21,0,,NW,M/S D. P. TRADERS,M/S D. P. TRADERS
20,40,Nanded Division,WASHIM,1,02-01-25,10:51,03-01-2025,NMSP,NMSP,DOC,...,BCN,21,21,21,21,0,,SC,M/S NARMADA SOLVEX PRIVATE LIMITED,M/S NARMADA SOLVEX PRIVATE LIMITED
24,5679,Nagaur,DONGAR GARH,1,02-01-25,14:50,03-01-2025,SGRI,SGRI,FG,...,BCN,42,42,42,42,0,,SEC,M/S SHRI GURUDEV RICE INUSTRIES,M/S SHRI GURUDEV RICE INUSTRIES


In [23]:
final_df_without_FCI['RAKE CMDT']

0     DOC
1      BR
2       W
3       W
4       W
5       W
15      W
16      W
20    DOC
24      R
25      M
27     BR
29      M
30      M
31      M
32      M
33      M
34      W
35      W
Name: RAKE CMDT, dtype: object

In [19]:

import asyncio
import pandas as pd
from telegram import Bot
from telegram.ext import ApplicationBuilder, ContextTypes

BOT_TOKEN = "7836500041:AAHOL2jJ8WGrRVeAnjJ3a354W6c6jgD22RU"
# CHAT_IDS = [8147978368, 499903657]  # Replace with your actual chat IDs
CHAT_IDS = [8147978368, 499903657, 7967517419, 7507991236]  # Replace with your actual chat IDs

async def send_daily_alert() -> None:
    # Assuming final_df_without_FCI is already defined and loaded elsewhere in the code
    try:
        # Filter the DataFrame for Rake Commodity 'M' and 'DOC'
        filtered_df = final_df_without_FCI[final_df_without_FCI['RAKE CMDT'].isin(['M', 'DOC'])]

        # Group and format the data, ensuring 'DVSN' is included
        grouped_info = filtered_df.groupby(
            ['DEMAND DATE', 'STTN FROM', 'DSTN', 'RAKE CMDT', 'CNSR_Full', 'CNSG_Full', 'DVSN', 'DEMAND TIME']
        ).size().reset_index(name='Count')

        # Prepare the message with improved formatting
        message = "*Daily Competitor Alert:*\n\n"  # Bold title and add space

        current_date = None  # To keep track of the current demand date in the loop
        for index, row in grouped_info.iterrows():
            if current_date != row['DEMAND DATE']:
                current_date = row['DEMAND DATE']
                message += f"*Demand Date:* {current_date}\n\n"  # Add Demand Date once per group

            message += (
                f"*From:* {row['STTN FROM']}\n"
                f"*To:* {row['DSTN']}\n"
                f"*Rake Commodity:* {row['RAKE CMDT']}\n"
                f"*Consigner:* {row['CNSR_Full']}\n"
                f"*Consignee:* {row['CNSG_Full']}\n"
                f"*Division:* {row['DVSN']}\n"  # Correctly included
                f"*Demand Time:* {row['DEMAND TIME']}\n"
                f"*Count:* {row['Count']}\n\n"
            )

        # Create bot instance and send message to each chat ID
        bot = Bot(token=BOT_TOKEN)
        for chat_id in CHAT_IDS:
            await bot.send_message(chat_id=chat_id, text=message, parse_mode='Markdown')  # Use Markdown for formatting

    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    # Check if there is an existing event loop
    try:
        loop = asyncio.get_event_loop()
        if loop.is_running():
            # If the loop is running, create a new task instead of using run()
            task = loop.create_task(send_daily_alert())
        else:
            asyncio.run(send_daily_alert())
    except RuntimeError as e:
        print(f"Error: {e}")



In [20]:
from datetime import datetime

# Get the current date and time
current_datetime = datetime.now()

# Format the date and time
formatted_datetime = current_datetime.strftime("%Y-%m-%d %H:%M:%S")

print("Current date and time:", formatted_datetime)


Current date and time: 2025-01-03 00:49:19
